In [1]:
import os
import json
import numpy as np
from itertools import permutations

In [4]:
# Configurations.
num_participants = 100
num_classes = 18
num_classes_per_visualization_per_gluten_type = 3
all_languages = ["English", "Russian", "No label"]
all_visualization_methods = ["Frame", "Baseline", "Scale"]

# Within session lists.
# Generate sample ID list and permutations.
# all_id_list = [int(item) for item in np.random.permutation(range(num_classes))]
# gluten_free_sample_id_list = all_id_list[:int(num_classes / 2)]
# not_gluten_free_sample_id_list = all_id_list[int(num_classes / 2):]
gluten_free_sample_id_list = [0, 2, 3, 5, 8, 10, 13, 17, 19]
not_gluten_free_sample_id_list = [1, 6, 9, 11, 12, 14, 15, 16, 18]
all_id_list = list(gluten_free_sample_id_list + not_gluten_free_sample_id_list)

all_gluten_free_sample_permutations = list(permutations(gluten_free_sample_id_list))
all_not_gluten_free_sample_permutations = list(permutations(not_gluten_free_sample_id_list))
np.random.shuffle(all_gluten_free_sample_permutations)
np.random.shuffle(all_not_gluten_free_sample_permutations)

print(all_id_list)
print(all_gluten_free_sample_permutations[:3])
print(all_not_gluten_free_sample_permutations[:3])

# Visualization list.
visualization_list = all_visualization_methods * num_classes_per_visualization_per_gluten_type
within_session_seq_len = len(visualization_list)
visualization_list

[0, 2, 3, 5, 8, 10, 13, 17, 19, 1, 6, 9, 11, 12, 14, 15, 16, 18]
[(10, 8, 2, 0, 19, 3, 13, 17, 5), (0, 5, 17, 3, 19, 10, 2, 8, 13), (17, 13, 8, 19, 2, 3, 5, 0, 10)]
[(6, 12, 18, 9, 11, 15, 1, 16, 14), (9, 14, 11, 1, 18, 16, 6, 12, 15), (18, 11, 16, 12, 9, 15, 1, 6, 14)]


['Frame',
 'Baseline',
 'Scale',
 'Frame',
 'Baseline',
 'Scale',
 'Frame',
 'Baseline',
 'Scale']

In [5]:
# Generate session sequences.
session_list = list(range(len(all_languages)))
all_session_sequences = list(permutations(session_list))
np.random.shuffle(all_session_sequences)
#all_session_sequences = all_session_sequences * int(np.ceil(num_participants / len(all_session_sequences)))
#all_session_sequences = all_session_sequences[:num_participants]
print(all_session_sequences)

[(2, 0, 1), (1, 0, 2), (2, 1, 0), (1, 2, 0), (0, 1, 2), (0, 2, 1)]


In [6]:
# Generate all participant sequences.
idx_sample_sequence = 0

all_participant_sequences = []
for idx_participant in range(num_participants):
    
    current_participant_sequence = []
    current_session_sequence = all_session_sequences[idx_participant % len(all_session_sequences)]
    
    for idx_session, idx_session_condition in enumerate(current_session_sequence[:1]):
        
        current_session_language_list = [all_languages[idx_session_condition]] * within_session_seq_len
        current_session_index_list = [idx_session+1] * within_session_seq_len
        
        # Calculate IDs with off-set.
#         id_offset = idx_session_condition * num_classes
        id_offset = 0
        current_gluten_free_samples = ["{:02d}".format(item + id_offset) for item in all_gluten_free_sample_permutations[idx_sample_sequence]]
        current_not_gluten_free_samples = ["{:02d}".format(item + id_offset) for item in all_not_gluten_free_sample_permutations[idx_sample_sequence]]
        
        # Get gluten-free and not-gluten-free samples. 
        current_gluten_free_sequence = list(zip(current_session_language_list, current_gluten_free_samples, visualization_list, current_session_index_list))
        current_not_gluten_free_sequence = list(zip(current_session_language_list, current_not_gluten_free_samples, visualization_list, current_session_index_list))
        
        # Combine and shuffle.
        current_session_sequence = current_gluten_free_sequence + current_not_gluten_free_sequence
        np.random.shuffle(current_session_sequence)
        
        # Append.
        current_participant_sequence += list(current_session_sequence)
        
        idx_sample_sequence += 1
        
    all_participant_sequences += [("P{:02d}".format(idx_participant+1), ) + item for item in current_participant_sequence]
    

In [9]:
# JSON.
with open("./sequences.json", "w") as f:
    json.dump(all_participant_sequences, f)
    
# CSV.
import pandas as pd
df = pd.DataFrame(all_participant_sequences, columns=["Participant", "Label language", "Sample id", "Visualization method", "Session number"])
with open("./sequences.csv", "w") as f:
    df.to_csv(f, index=False, line_terminator="\n")

In [8]:
all_participant_sequences

[('P01', 'No label', '05', 'Scale', 1),
 ('P01', 'No label', '10', 'Frame', 1),
 ('P01', 'No label', '11', 'Baseline', 1),
 ('P01', 'No label', '01', 'Frame', 1),
 ('P01', 'No label', '13', 'Frame', 1),
 ('P01', 'No label', '06', 'Frame', 1),
 ('P01', 'No label', '18', 'Scale', 1),
 ('P01', 'No label', '14', 'Scale', 1),
 ('P01', 'No label', '19', 'Baseline', 1),
 ('P01', 'No label', '17', 'Baseline', 1),
 ('P01', 'No label', '12', 'Baseline', 1),
 ('P01', 'No label', '08', 'Baseline', 1),
 ('P01', 'No label', '16', 'Baseline', 1),
 ('P01', 'No label', '03', 'Scale', 1),
 ('P01', 'No label', '09', 'Frame', 1),
 ('P01', 'No label', '00', 'Frame', 1),
 ('P01', 'No label', '15', 'Scale', 1),
 ('P01', 'No label', '02', 'Scale', 1),
 ('P02', 'Russian', '17', 'Scale', 1),
 ('P02', 'Russian', '00', 'Frame', 1),
 ('P02', 'Russian', '09', 'Frame', 1),
 ('P02', 'Russian', '18', 'Baseline', 1),
 ('P02', 'Russian', '10', 'Scale', 1),
 ('P02', 'Russian', '01', 'Frame', 1),
 ('P02', 'Russian', '06',